In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': 'e22daf50a33f',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'VVaP-BPiSyKO_yho-aX1Hg',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [2]:
from datasets import load_dataset

dataset = load_dataset("nlplabtdtu/edu-crawl-with-date", split="train[:500]" , token="")

/opt/conda/envs/elastic/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('nlplabtdtu/bert-70M-cased-senformer')

In [4]:
from elasticsearch import helpers

In [ ]:
if not es.indices.exists(index="edu_data_index"):
    try:
      es_index = {
        "mappings": {
          "properties": {
            "title": {
              "type": "text"
            },
            "body": {
              "type": "text"
            },
            "date": {
              "type": "text"
            },
            "body_vector": {
              "type": "dense_vector",
              "dims": 768
            }
          }
        }
      }

      es.indices.create(index='edu_data_index', body=es_index, ignore=[400])
      bulk_data = []
      for i in range(len(dataset)):
        embedding = model.encode(dataset[i]['body'], show_progress_bar=False)
        bulk_data.append({
                "_index": 'edu_data_index',
                "_source": {
                    "title": dataset[i]['title'],
                    "body": dataset[i]['body'],
                    "date": dataset[i]['date'],
                    "body_vector": embedding
                }
            })
      print(bulk_data[0])

      helpers.bulk(es, bulk_data)

    except:
        print("During index an exception occured. Continue\n\n")

In [5]:
es.indices.refresh(index="edu_data_index")
es.cat.count(index="edu_data_index", format="json")

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [edu_data_index]', edu_data_index, index_or_alias)

In [6]:
inp_question = """Ban Giám đốc đổi tên Ban Khảo thí và Kiểm định chất lượng đào tạo thành Ban Khảo thí và Quản lý chất lượng vào thời gian nào?"""

In [10]:
import time 

encode_start_time = time.time()
question_embedding = model.encode(inp_question)
encode_end_time = time.time()

print("Encoding time:", encode_end_time - encode_start_time)

Encoding time: 0.029086589813232422


In [7]:
bm25 = es.search(
    index="edu_data_index", 
    body={"query": 
          {"match": {"body": inp_question }}
    }
)

print("Searching time:", bm25['took']/1000)
print("BM25 results:")
for hit in bm25['hits']['hits'][0:3]:
    print("\t{}".format(hit['_source']['body'][:500]))

Searching time: 0.012
BM25 results:
	
Thứ ba, 22/11/2022 - 9:2
BAN KHẢO THÍ VÀ QUẢN LÝ CHẤT LƯỢNG
Sau 4 năm hoạt động, để phù hợp với tình hình thực hiện chức năng, nhiệm vụ, Giám đốc Học viện đã đổi tên Ban Khảo thí và Kiểm định chất lượng đào tạo thành Ban Khảo thí và Quản lý chất lượng vào tháng 10 năm 2010.
I. Liên hệ
- Tên đơn vị: Ban Khảo thí và Quản lý chất lượng
- Địa chỉ: Số 58 Lê Văn Hiến, Phường Đức Thắng, Quận Bắc Từ Liêm, Hà Nội
- Số điện thoại liên hệ: 
- Email: khaothi_qlcl@hvtc.edu.vn
II. Quá trình thành lập và phát
	
Thứ ba, 22/11/2022 - 9:2
BAN KHẢO THÍ VÀ QUẢN LÝ CHẤT LƯỢNG
Sau 4 năm hoạt động, để phù hợp với tình hình thực hiện chức năng, nhiệm vụ, Giám đốc Học viện đã đổi tên Ban Khảo thí và Kiểm định chất lượng đào tạo thành Ban Khảo thí và Quản lý chất lượng vào tháng 10 năm 2010.
I. Liên hệ
- Tên đơn vị: Ban Khảo thí và Quản lý chất lượng
- Địa chỉ: Số 58 Lê Văn Hiến, Phường Đức Thắng, Quận Bắc Từ Liêm, Hà Nội
- Số điện thoại liên hệ: 
- Email: khaothi_qlcl@hvtc

/tmp/ipykernel_14190/3452357257.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  bm25 = es.search(


In [14]:
sem_search = es.search(index="edu_data_index", body=
                       {
                            "query": {
                                "script_score": {
                                    "query" : {
                                        "match_all": {},
                                    },
                                    "script": {
                                        "source": "cosineSimilarity(params.query_vector, 'body_vector') + 1.0", 
                                        "params": {
                                            "query_vector": question_embedding
                                        }
                                    }
                                }
                            }
                        }
)

print("Searching time:", sem_search['took']/1000)

print("\nSemantic Search results:")
for hit in sem_search['hits']['hits'][0:3]:
    print("\t{}".format(hit['_source']['body'][:500]))

Searching time: 0.011

Semantic Search results:
	
Thứ ba, 22/11/2022 - 9:2
BAN KHẢO THÍ VÀ QUẢN LÝ CHẤT LƯỢNG
Sau 4 năm hoạt động, để phù hợp với tình hình thực hiện chức năng, nhiệm vụ, Giám đốc Học viện đã đổi tên Ban Khảo thí và Kiểm định chất lượng đào tạo thành Ban Khảo thí và Quản lý chất lượng vào tháng 10 năm 2010.
I. Liên hệ
- Tên đơn vị: Ban Khảo thí và Quản lý chất lượng
- Địa chỉ: Số 58 Lê Văn Hiến, Phường Đức Thắng, Quận Bắc Từ Liêm, Hà Nội
- Số điện thoại liên hệ: 
- Email: khaothi_qlcl@hvtc.edu.vn
II. Quá trình thành lập và phát
	
Thứ ba, 22/11/2022 - 9:2
BAN KHẢO THÍ VÀ QUẢN LÝ CHẤT LƯỢNG
Sau 4 năm hoạt động, để phù hợp với tình hình thực hiện chức năng, nhiệm vụ, Giám đốc Học viện đã đổi tên Ban Khảo thí và Kiểm định chất lượng đào tạo thành Ban Khảo thí và Quản lý chất lượng vào tháng 10 năm 2010.
I. Liên hệ
- Tên đơn vị: Ban Khảo thí và Quản lý chất lượng
- Địa chỉ: Số 58 Lê Văn Hiến, Phường Đức Thắng, Quận Bắc Từ Liêm, Hà Nội
- Số điện thoại liên hệ: 
- Email: khaot

/tmp/ipykernel_2147/4175631082.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  sem_search = es.search(index="edu_data_index", body=


In [38]:
eval_dataset = load_dataset("nlplabtdtu/xquad_benchmark",split='train')

In [39]:
eval_dataset

Dataset({
    features: ['question', 'contexts'],
    num_rows: 1190
})

In [40]:
unique_context = []
for row in eval_dataset:
    # print(row)
    for context in row['contexts']:
        unique_context.append(context)
print(len(unique_context))
unique_context = list(set(unique_context))
print(len(unique_context))

5950
240


In [50]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nlplabtdtu/bert-30M-uncased-senformer')

In [53]:
es.indices.delete(index="eval_data_index", ignore=[400])

/tmp/ipykernel_14190/411385408.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="eval_data_index", ignore=[400])


ObjectApiResponse({'acknowledged': True})

In [54]:
if not es.indices.exists(index="eval_data_index"):
    # try:
      es_index = {
        "mappings": {
          "properties": {

            "body": {
              "type": "text"
            },
            "body_vector": {
              "type": "dense_vector",
              "dims": 512
            }
          }
        }
      }

      es.indices.create(index='eval_data_index', body=es_index, ignore=[400])
      bulk_data = []
      for i in range(len(unique_context)):
        embedding = model.encode(unique_context[i], show_progress_bar=False)
        bulk_data.append({
                "_index": 'eval_data_index',
                "_source": {
                    "body": unique_context[i],
                    "body_vector": embedding
                }
            })
      # print(bulk_data[0])

      helpers.bulk(es, bulk_data)

    # except:
    #     print("During index an exception occured. Continue\n\n")

/tmp/ipykernel_14190/2657275120.py:18: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index='eval_data_index', body=es_index, ignore=[400])
/tmp/ipykernel_14190/2657275120.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.indices.create(index='eval_data_index', body=es_index, ignore=[400])


In [48]:
import time
start_time = time.time()
count_true = 0
for row in eval_dataset:
    inp_question = row['question']
    bm25 = es.search(
        index="eval_data_index", 
        body={"query": 
            {"match": {"body": inp_question }}
        },
        size=1
    )
    if bm25['hits']['hits'][0]['_source']['body'] == row['contexts'][-1]:
        count_true += 1
end_time = time.time()
print("Total time:", end_time - start_time)
print("Accuracy:", count_true/len(eval_dataset))

/tmp/ipykernel_14190/2152021411.py:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  bm25 = es.search(


Total time: 10.321417570114136
Accuracy: 0.9168067226890756


In [57]:
import time
start_time = time.time()
count_true = 0
for row in eval_dataset:
    inp_question = row['question']
    question_embedding = model.encode(inp_question)
    sem_search = es.search(index="eval_data_index", body=
                       {
                            "query": {
                                "script_score": {
                                    "query" : {
                                        "match_all": {},
                                    },
                                    "script": {
                                        "source": "cosineSimilarity(params.query_vector, 'body_vector') + 1.0", 
                                        "params": {
                                            "query_vector": question_embedding
                                        }
                                    }
                                }
                            }
                        },
                        size=5
    )
    for hit in sem_search['hits']['hits']:
        if hit['_source']['body'] == row['contexts'][-1]:
            count_true += 1
            break
end_time = time.time()
print("Total time:", end_time - start_time)
print("Accuracy:", count_true/len(eval_dataset))

/tmp/ipykernel_14190/2443178705.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  sem_search = es.search(index="eval_data_index", body=


Total time: 27.303131818771362
Accuracy: 0.9949579831932773


In [56]:
import time
import torch
start_time = time.time()
count_true = 0
for row in eval_dataset:
    inp_question = row['question']
    bm25 = es.search(
        index="eval_data_index", 
        body={"query": 
            {"match": {"body": inp_question }}
        },
        size=2
    )

    encoded_contexts = [hit['_source']['body_vector'] for hit in bm25['hits']['hits']]
    encoded_contexts = torch.tensor(encoded_contexts)
    contexts = [hit['_source']['body'] for hit in bm25['hits']['hits']]
    result = util.semantic_search(model.encode(inp_question), encoded_contexts, top_k=1)
    # print(result)
    if contexts[int(result[0][0]['corpus_id'])] == row['contexts'][-1]:
        count_true += 1
    # else:
    #     print(f"predict: {contexts[int(result[0][0]['corpus_id'])]}")
    #     print(contexts)
    #     print(inp_question)
    #     break
end_time = time.time()
print("Total time:", end_time - start_time)
print("Accuracy:", count_true/len(eval_dataset))

/tmp/ipykernel_14190/196540366.py:7: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  bm25 = es.search(


Total time: 26.23408532142639
Accuracy: 0.9100840336134454
